#Web Scraping


In [1]:
import requests
import json
from bs4 import BeautifulSoup


In [24]:
page_url = 'https://www.pudelek.pl/kate-middleton-w-kolczykach-od-krolowej-elzbiety-oddaje-hold-zmarlej-monarchini-u-boku-ksiecia-williama-zdjecia-6812680076917504g/'
page = requests.get(page_url)

In [25]:
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="pl">
 <head>
  <meta charset="utf-8" class="next-head"/>
  <link class="next-head" href="/static/favicon.ico" rel="shortcut icon"/>
  <link class="next-head" href="/static/opensearch.xml" rel="search" title="Pudelek" type="application/opensearchdescription+xml"/>
  <link class="next-head" href="/static/apple-touch-icon-60x60.png" rel="apple-touch-icon"/>
  <link class="next-head" href="/static/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link class="next-head" href="/static/apple-touch-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
  <link class="next-head" href="/static/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link class="next-head" href="/static/apple-touch-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
  <link class="next-head" href="/static/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link class="next-head" href="/static/apple-touch-icon-120x120.png" re

In [27]:
clas = "sc-1mskw74-0 sc-7eqdwf-0 sc-q1w81m-0 cyNurK"
comm = soup.find_all("div", {"class": clas})
print(comm[4].get_text())
print(len(comm))

Dobrze wygląda 
19


**HTML file does not contain all comments, padding is inteactive and does not chcenge the url adress, so there is a need to use API through GraphQL.**

In [22]:
#first 5 comments
data = {"query":"query comments(\n  $productId: String!\n  $materialId: String!\n  $parentId: String\n  $offset: Int = 0\n  $first: Int = 10\n  $sort: CommentsSortType = LATEST\n  $popular: Boolean = false\n  $popularFirst: Int = 3\n) {\n  comments: comments(\n    productId: $productId\n    contentId: $materialId\n    parentId: $parentId\n    offset: $offset\n    first: $first\n    sort: $sort\n  ) {\n    ...commentEdges\n    pageInfo {\n      hasNextPage\n      hasPreviousPage\n      endCursor\n      totalCount\n    }\n  }\n  popular: comments(\n    productId: $productId\n    contentId: $materialId\n    parentId: $parentId\n    offset: 0\n    first: $popularFirst\n    sort: TOP_RATED\n  ) @include(if: $popular) {\n    ...commentEdges\n  }\n}\n\nfragment commentEdges on CommentsConnection {\n  edges {\n    node {\n      id\n      text\n      nick\n      created\n      modified\n      votesUp\n      votesDown\n      replies\n    }\n  }\n}\n",
        "variables":{"productId":"5980322589471361","materialId":"6812680076917504","offset":5,"first":5,"sort":"LATEST","popularFirst":5,"popular":False}}

response = requests.post('https://data-api.wp.pl/graphql', json=data)
print(json.dumps(response.json(), indent=2))

{
  "data": {
    "comments": {
      "edges": [
        {
          "node": {
            "id": "6812708801016386",
            "text": "Prosz\u0119 Pana redaktora-p\u0142aszcz si\u0119ga po\u0142owy \u0142ydki A nie ziemi ",
            "nick": "Kkk",
            "created": 1663258881,
            "modified": 1663258881,
            "votesUp": 10,
            "votesDown": 0,
            "replies": 0
          }
        },
        {
          "node": {
            "id": "6812708015469121",
            "text": "Oni nie patrz\u0105 sobie w oczy, a jak j\u0105 dotkn\u0105 to tak drgn\u0119\u0142a \u017ce huragan we w\u0142osy zawia\u0142   -masakra!!!",
            "nick": "wer",
            "created": 1663258673,
            "modified": 1663259349,
            "votesUp": 7,
            "votesDown": 7,
            "replies": 1
          }
        },
        {
          "node": {
            "id": "6812705979725953",
            "text": "Wygl\u0105da na zm\u0119czon\u0105 ",
            "

In [23]:
soup = BeautifulSoup(response.content, 'html.parser') #to get Polish letters

print(soup.prettify())

{"data":{"comments":{"edges":[{"node":{"id":"6812708801016386","text":"Proszę Pana redaktora-płaszcz sięga połowy łydki A nie ziemi ","nick":"Kkk","created":1663258881,"modified":1663258881,"votesUp":10,"votesDown":0,"replies":0}},{"node":{"id":"6812708015469121","text":"Oni nie patrzą sobie w oczy, a jak ją dotkną to tak drgnęła że huragan we włosy zawiał   -masakra!!!","nick":"wer","created":1663258673,"modified":1663259349,"votesUp":7,"votesDown":7,"replies":1}},{"node":{"id":"6812705979725953","text":"Wygląda na zmęczoną ","nick":"amc","created":1663258207,"modified":1663258207,"votesUp":9,"votesDown":1,"replies":0}},{"node":{"id":"6812705462302337","text":"jak kolczykami można złożyć hołd??","nick":"dsaf","created":1663258073,"modified":1663258073,"votesUp":17,"votesDown":2,"replies":0}},{"node":{"id":"6812705193792577","text":"ci wszyscy co mówią złe Kate i William nie okazują sobie uczuć podczas publicznych wyjść - to jest protokół ! oni zdają sobie sprawę, ze są twarzą monarchi